In [1]:
import numpy as np
from scipy.signal import butter, lfilter
import matplotlib.pyplot as plt
from PIL import Image
import librosa
import os
import soundfile as sf

In [2]:
import soundfile as sf
def preprocess(file,path):
    data, sample_rate = librosa.load(file) 
    cutoff_freq = 195  # Set the cutoff frequency in Hz
    sampling_freq = sample_rate  # Set the sampling frequency in Hz
    normalized_cutoff = cutoff_freq / (0.5 * sampling_freq) # denominator=nyquist frequency

    # Design the Butterworth
    order = 4  # Filter order
    b, a = butter(order, normalized_cutoff, btype='low',analog=False)

    #zero-phase filter
    filtered_data = lfilter(b, a, data)
    
    # Define the target sample rate for downsampling
    target_sample_rate = sample_rate//10

    # Downsample the audio data
    downsampled_data = librosa.resample(filtered_data, orig_sr=sampling_freq, target_sr=target_sample_rate)

    # Save the downsampled audio to a new file
    downsampled_file=f"trash2\\{file[-16:]}.wav"
    sf.write(downsampled_file, downsampled_data, target_sample_rate)
    
    # Load the downsampled audio file
    downsampled_data, sample_rate = sf.read(downsampled_file)

    # Calculate the desired duration of each segment (3 seconds)
    segment_duration = 3  # in seconds

    # Calculate the number of samples per segment
    samples_per_segment = int(segment_duration * target_sample_rate)

    # Split the audio data into segments
    segments = []
    num_segments = len(downsampled_data) // samples_per_segment
    for i in range(num_segments):
        segment = downsampled_data[i * samples_per_segment:(i + 1) * samples_per_segment]
        segments.append(segment)

    # Save each segment as a separate audio file
    for i, segment in enumerate(segments):
        segment_file =f"SplitA\\{path}seg{i}_{file[-16:]}.wav"
        sf.write(segment_file, segment, target_sample_rate)
        
    # Assuming you have the segmented audio files saved as 'segment_1.wav', 'segment_2.wav', etc.

    # Define the pitch shift and time shift parameters
    pitch_shift_semitones = -0.5  # Shift by one half step
    time_shift_units = 1  # Shift by 1 unit (in seconds)

    # Iterate over each segment
    for i in range(num_segments):
        segment_file =f"SplitA\\{path}seg{i}_{file[-16:]}.wav"
        segment_data, sample_rate = sf.read(segment_file)

        # Apply pitch shift
        pitch_shifted_audio = librosa.effects.pitch_shift(segment_data, sr=target_sample_rate, n_steps=-0.5)
        augmented_segment_file = f"AugSegA\\{path}pitch_aug{segment_file[-25:]}"
        sf.write(augmented_segment_file, pitch_shifted_audio, target_sample_rate)   
    
        # Apply time shift
        time_shifted_data = librosa.effects.time_stretch(segment_data, rate=1.0 / time_shift_units)

        # Save augmented segment as a new audio file
        augmented_segment_file = f"AugSegA\\{path}time_aug{segment_file[-25:]}"
        sf.write(augmented_segment_file, time_shifted_data, target_sample_rate)     

In [3]:
import os
# Specify the folder path containing the audio files
folder_path = r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\Atraining_normal"
path=f"Normal\\"

# Get a list of all files in the folder
files = os.listdir(folder_path)

# Iterate over each file in the folder
for file in files:
    file_path = os.path.join(folder_path, file)
#     print(file_path[-16:])
    preprocess(file_path,path)

In [4]:
# Specify the folder path containing the audio files
folder_path = r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\Atraining_murmur"
path=f"Murmur\\"

# Get a list of all files in the folder
files = os.listdir(folder_path)

# Iterate over each file in the folder
for file in files:
    file_path = os.path.join(folder_path, file)
#     print(file_path[-16:])
    preprocess(file_path,path)

In [5]:
# Specify the folder path containing the audio files
folder_path = r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\Atraining_artifact"
path=f"Artifact\\"

# Get a list of all files in the folder
files = os.listdir(folder_path)

# Iterate over each file in the folder
for file in files:
    file_path = os.path.join(folder_path, file)
#     print(file_path[-16:])
    preprocess(file_path,path)

In [6]:
# Specify the folder path containing the audio files
folder_path = r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\Atraining_extrahs"
path=f"Extrahs\\"

# Get a list of all files in the folder
files = os.listdir(folder_path)

# Iterate over each file in the folder
for file in files:
    file_path = os.path.join(folder_path, file)
#     print(file_path[-16:])
    preprocess(file_path,path)

In [7]:
extracted_features=[]

In [8]:
target_size=(128,128)
def toSpec(file,path):
    y, sr = librosa.load(file)
    # Compute spectrogram
    targetsr=sr//10
    spectrogram = librosa.feature.melspectrogram(y=y, sr=targetsr)

    # Convert to decibel scale
    spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)

    fig, ax = plt.subplots(figsize=(1.28,1.28))
    librosa.display.specshow(spectrogram_db, sr=targetsr, fmax=8000)
    savename=f"Trashimg\\spec{file[-33:]}.png"
    plt.savefig(savename,transparent=True)
    plt.close()

    img = Image.open(savename).convert('RGB')
    img.save(f"SpecA\\{path}\\spec{file[-33:]}.png")
    img_array = np.array(img)
    img_array = img_array / 255.0
    img_array = np.array(img_array)
    extracted_features.append([img_array,path])

In [9]:
folder_path = r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\AugSegA\Normal"
path=f"Normal"
# Get a list of all files in the folder
files = os.listdir(folder_path)

# Iterate over each file in the folder
for file in files:
    file_path = os.path.join(folder_path, file)
#     print(file_path)
    toSpec(file_path,path)

In [10]:
folder_path = r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\AugSegA\Murmur"
path=f"Murmur"
# Get a list of all files in the folder
files = os.listdir(folder_path)

# Iterate over each file in the folder
for file in files:
    file_path = os.path.join(folder_path, file)
#     print(file_path[-20:])
    toSpec(file_path,path)

In [11]:
folder_path = r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\AugSegA\Extrahs"
path=f"Extrahs"
# Get a list of all files in the folder
files = os.listdir(folder_path)

# Iterate over each file in the folder
for file in files:
    file_path = os.path.join(folder_path, file)
#     print(file_path[-20:])
    toSpec(file_path,path)

In [12]:
folder_path = r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\AugSegA\Artifact"
path=f"Artifact"
# Get a list of all files in the folder
files = os.listdir(folder_path)

# Iterate over each file in the folder
for file in files:
    file_path = os.path.join(folder_path, file)
#     print(file_path[-20:])
    toSpec(file_path,path)

In [13]:
import pandas as pd
df=pd.DataFrame(extracted_features,columns=['feature','class'])
df.head()

,feature,class
0,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",Normal
1,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",Normal
2,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",Normal
3,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",Normal
4,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",Normal


In [14]:
x=np.array(df['feature'].tolist())
y=np.array(df['class'].tolist())

In [15]:
x.shape

(582, 128, 128, 3)

In [16]:
y.shape

(582,)

In [17]:
dfcopy=df
dfcopy.shape

(582, 2)

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import cv2

def time_warp(spec, W=5):
    num_bins, num_frames = spec.shape
    spec_aug = np.copy(spec)

    # Generate random control points for time warping
    control_points = [(0, 0), (num_bins-1, num_frames-1)]
    for _ in range(W):
        random_point = np.random.randint(1, num_bins-1)
        control_points.append((random_point, np.random.randint(0, num_frames-1)))

    # Apply time warping
    for i in range(len(control_points)-1):
        c1, c2 = control_points[i], control_points[i+1]
        spec_aug[c1[0]:c2[0]+1, :] = np.roll(spec[c1[0]:c2[0]+1, :], c2[1] - c1[1], axis=1)

    return spec_aug

def frequency_mask(spec, F=30, num_masks=2):
    spec_aug = np.copy(spec)

    # Apply frequency masking
    num_bins, num_frames = spec.shape
    for _ in range(num_masks):
        f = np.random.randint(0, F)
        f0 = np.random.randint(0, num_bins - f)
        spec_aug[f0:f0+f, :] = 0

    return spec_aug

In [19]:
#SpecAugment-A

def SpecAugm(filename, path):
    audio, sr = librosa.load(filename, sr=None)
    sr=sr//10;
    # Compute spectrogram
    spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=2048, hop_length=512, n_mels=128)
    spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)
    
    # Normalize the spectrogram to values between 0 and 1
    spectrogram_normalized = (spectrogram_db - np.min(spectrogram_db)) / (np.max(spectrogram_db) - np.min(spectrogram_db))
    # Scale the spectrogram to values between 1 and 255
    spectrogram_scaled = spectrogram_normalized * 255
    
    fig, ax = plt.subplots(figsize=(1.28,1.28))
    
    img4=librosa.display.specshow(spectrogram_scaled, sr=sr)
    plt.savefig((r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\SpecAugA\{}\{}_spectogrm.png").format(path,filename[-25:]), transparent=True)
    plt.close()
                
    # Apply time warping
    time_warped_spectrogram = time_warp(spectrogram_scaled)

    # Generate frequency-masked spectrograms
    frequency_masked_spectrogram_1 = frequency_mask(spectrogram_scaled)
    frequency_masked_spectrogram_2 = frequency_mask(spectrogram_scaled)
    
    fig, ax = plt.subplots(figsize=(1.28,1.28))
                
    img1=librosa.display.specshow(time_warped_spectrogram, sr=sr)
    plt.savefig((r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\SpecAugA\{}\{}_timewrap.png").format(path,filename[-25:]), transparent=True)
    plt.close()
     
    fig, ax = plt.subplots(figsize=(1.28,1.28))
    
    img2=librosa.display.specshow(frequency_masked_spectrogram_1, sr=sr)
    plt.savefig((r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\SpecAugA\{}\{}_freqwrap1.png").format(path,filename[-25:]), transparent=True)
    plt.close()
    
     
    fig, ax = plt.subplots(figsize=(1.28,1.28))
    
    img3=librosa.display.specshow(frequency_masked_spectrogram_2, sr=sr)
    plt.savefig((r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\SpecAugA\{}\{}_freqwrap2.png").format(path,filename[-25:]), transparent=True)
    plt.close()

In [20]:
folder_path = r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\SplitA\Normal"
path=f"Normal"
# Get a list of all files in the folder
files = os.listdir(folder_path)

# Iterate over each file in the folder
for file in files:
    file_path = os.path.join(folder_path, file)
#     print(file_path[-25:])
    SpecAugm(file_path,path)

In [21]:
folder_path = r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\SplitA\Murmur"
path=f"Murmur"
# Get a list of all files in the folder
files = os.listdir(folder_path)

# Iterate over each file in the folder
for file in files:
    file_path = os.path.join(folder_path, file)
#     print(file_path[-25:])
    SpecAugm(file_path,path)

In [22]:
folder_path = r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\SplitA\Extrahs"
path=f"Extrahs"
# Get a list of all files in the folder
files = os.listdir(folder_path)

# Iterate over each file in the folder
for file in files:
    file_path = os.path.join(folder_path, file)
#     print(file_path[-25:])
    SpecAugm(file_path,path)

In [23]:
folder_path = r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\SplitA\Artifact"
path=f"Artifact"
# Get a list of all files in the folder
files = os.listdir(folder_path)

# Iterate over each file in the folder
for file in files:
    file_path = os.path.join(folder_path, file)
#     print(file_path[-25:])
    SpecAugm(file_path,path)

In [24]:
extracted_features1=[]
folder_path = r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\SpecAugA\Normal"
path=f"Normal"
# Get a list of all files in the folder
files = os.listdir(folder_path)

# Iterate over each file in the folder
for file in files:
    file_path = os.path.join(folder_path, file)
#     print(file_path)
    img = Image.open(file_path).convert('RGB')
    img_array = np.array(img)
    img_array = img_array / 255.0
    img_array = np.array(img_array)
    extracted_features1.append([img_array,path])

In [25]:
folder_path = r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\SpecAugA\Murmur"
path=f"Murmur"
# Get a list of all files in the folder
files = os.listdir(folder_path)

# Iterate over each file in the folder
for file in files:
    file_path = os.path.join(folder_path, file)
#     print(file_path)
    img = Image.open(file_path).convert('RGB')
    img_array = np.array(img)
    img_array = img_array / 255.0
    img_array = np.array(img_array)
    extracted_features1.append([img_array,path])

In [26]:
folder_path = r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\SpecAugA\Extrahs"
path=f"Extrahs"
# Get a list of all files in the folder
files = os.listdir(folder_path)

# Iterate over each file in the folder
for file in files:
    file_path = os.path.join(folder_path, file)
#     print(file_path)
    img = Image.open(file_path).convert('RGB')
    img_array = np.array(img)
    img_array = img_array / 255.0
    img_array = np.array(img_array)
    extracted_features1.append([img_array,path])

In [27]:
folder_path = r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\SpecAugA\Artifact"
path=f"Artifact"
# Get a list of all files in the folder
files = os.listdir(folder_path)

# Iterate over each file in the folder
for file in files:
    file_path = os.path.join(folder_path, file)
#     print(file_path)
    img = Image.open(file_path).convert('RGB')
    img_array = np.array(img)
    img_array = img_array / 255.0
    img_array = np.array(img_array)
    extracted_features1.append([img_array,path])

In [28]:
df1=pd.DataFrame(extracted_features1,columns=['feature','class'])
df1.head()

,feature,class
0,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",Normal
1,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",Normal
2,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",Normal
3,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",Normal
4,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",Normal


In [29]:
x=np.array(df1['feature'].tolist())
y=np.array(df1['class'].tolist())

In [30]:
x.shape

(1164, 128, 128, 3)

In [31]:
y.shape

(1164,)

In [32]:
df1copy=df1
df1copy.shape

(1164, 2)

In [33]:
df_main = pd.concat([df, df1], ignore_index=True)
df_main

,feature,class
0,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",Normal
1,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",Normal
2,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",Normal
3,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",Normal
4,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",Normal
...,...,...
1741,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",Artifact
1742,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",Artifact
1743,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",Artifact
1744,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",Artifact


In [34]:
df_maincopy=df_main
df_maincopy.shape

(1746, 2)

In [35]:
# Convert DataFrame to CSV
df_main.to_csv('heartaudio.csv', index=False)

In [36]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1746 entries, 0 to 1745
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   feature  1746 non-null   object
 1   class    1746 non-null   object
dtypes: object(2)
memory usage: 27.4+ KB


In [37]:
df_main['class'].value_counts()

class
Artifact    720
Normal      408
Murmur      396
Extrahs     222
Name: count, dtype: int64

In [38]:
x=np.array(df_main['feature'].tolist())
y=np.array(df_main['class'].tolist())

In [39]:
x.shape

(1746, 128, 128, 3)

In [40]:
y.shape

(1746,)

In [42]:
dfx = df_main.drop(df_main[df_main['class'] == 'Artifact'].sample(frac=0.5).index)
# dfx.shape
dfx['class'].value_counts()

class
Normal      408
Murmur      396
Artifact    360
Extrahs     222
Name: count, dtype: int64

In [43]:
df2=dfx
df2['class'].value_counts()

class
Normal      408
Murmur      396
Artifact    360
Extrahs     222
Name: count, dtype: int64

In [44]:
df2copy=df2
df2copy.shape

(1386, 2)

In [45]:
x=np.array(df2['feature'].tolist())
y=np.array(df2['class'].tolist())

In [46]:
x=np.concatenate((x, x), axis=0)
y=np.concatenate((y, y), axis=0)

In [47]:
x.shape

(2772, 128, 128, 3)

In [48]:
y.shape

(2772,)

In [49]:
y

array(['Normal', 'Normal', 'Normal', ..., 'Artifact', 'Artifact',
       'Artifact'], dtype='<U8')

In [50]:
y=np.array(pd.get_dummies(y))
y

array([[False, False, False,  True],
       [False, False, False,  True],
       [False, False, False,  True],
       ...,
       [ True, False, False, False],
       [ True, False, False, False],
       [ True, False, False, False]])

In [51]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [52]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

In [53]:
X_train.shape

(1884, 128, 128, 3)

In [54]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD

# Load your data and labels

# Create an instance of ResNet152V2 model without the top (classification) layer
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False
    
# Add custom layers on top of the pre-trained model
flatten_layer = Flatten()(base_model.output)
dense_layer = Dense(128, activation='relu')(flatten_layer)
dropout_layer = Dropout(0.25)(dense_layer)
classification_layer = Dense(4, activation='softmax')(dropout_layer)  # Replace 'num_classes' with the number of classes in your problem

# Create the final model
model = Model(inputs=base_model.input, outputs=classification_layer) 

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [55]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 66, 66, 64)   0           ['conv1_conv[0][0]']         

 vation)                                                                                          
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 32, 32, 64)   16384       ['conv2_block3_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv2_block3_1_bn (BatchNormal  (None, 32, 32, 64)  256         ['conv2_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 32, 32, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
          

 conv3_block2_2_bn (BatchNormal  (None, 16, 16, 128)  512        ['conv3_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block2_2_relu (Activatio  (None, 16, 16, 128)  0          ['conv3_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_3_conv (Conv2D)   (None, 16, 16, 512)  66048       ['conv3_block2_2_relu[0][0]']    
                                                                                                  
 conv3_block2_out (Add)         (None, 16, 16, 512)  0           ['conv3_block1_out[0][0]',       
                                                                  'conv3_block2_3_conv[0][0]']    
          

 ization)                                                                                         
                                                                                                  
 conv3_block5_1_relu (Activatio  (None, 16, 16, 128)  0          ['conv3_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block5_2_pad (ZeroPaddin  (None, 18, 18, 128)  0          ['conv3_block5_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv3_block5_2_conv (Conv2D)   (None, 16, 16, 128)  147456      ['conv3_block5_2_pad[0][0]']     
                                                                                                  
 conv3_blo

 conv3_block8_preact_bn (BatchN  (None, 16, 16, 512)  2048       ['conv3_block7_out[0][0]']       
 ormalization)                                                                                    
                                                                                                  
 conv3_block8_preact_relu (Acti  (None, 16, 16, 512)  0          ['conv3_block8_preact_bn[0][0]'] 
 vation)                                                                                          
                                                                                                  
 conv3_block8_1_conv (Conv2D)   (None, 16, 16, 128)  65536       ['conv3_block8_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv3_block8_1_bn (BatchNormal  (None, 16, 16, 128)  512        ['conv3_block8_1_conv[0][0]']    
 ization) 

...

 conv4_block29_1_bn (BatchNorma  (None, 8, 8, 256)   1024        ['conv4_block29_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block29_1_relu (Activati  (None, 8, 8, 256)   0           ['conv4_block29_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block29_2_pad (ZeroPaddi  (None, 10, 10, 256)  0          ['conv4_block29_1_relu[0][0]']   
 ng2D)                                                                                            
                                                                                                  
 conv4_block29_2_conv (Conv2D)  (None, 8, 8, 256)    589824      ['conv4_block29_2_pad[0][0]']    
          

                                                                                                  
 conv4_block32_preact_bn (Batch  (None, 8, 8, 1024)  4096        ['conv4_block31_out[0][0]']      
 Normalization)                                                                                   
                                                                                                  
 conv4_block32_preact_relu (Act  (None, 8, 8, 1024)  0           ['conv4_block32_preact_bn[0][0]']
 ivation)                                                                                         
                                                                                                  
 conv4_block32_1_conv (Conv2D)  (None, 8, 8, 256)    262144      ['conv4_block32_preact_relu[0][0]
                                                                 ']                               
                                                                                                  
 conv4_blo

 conv4_block34_2_bn (BatchNorma  (None, 8, 8, 256)   1024        ['conv4_block34_2_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block34_2_relu (Activati  (None, 8, 8, 256)   0           ['conv4_block34_2_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block34_3_conv (Conv2D)  (None, 8, 8, 1024)   263168      ['conv4_block34_2_relu[0][0]']   
                                                                                                  
 conv4_block34_out (Add)        (None, 8, 8, 1024)   0           ['conv4_block33_out[0][0]',      
                                                                  'conv4_block34_3_conv[0][0]']   
          

                                                                                                  
 conv5_block1_1_bn (BatchNormal  (None, 4, 4, 512)   2048        ['conv5_block1_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block1_1_relu (Activatio  (None, 4, 4, 512)   0           ['conv5_block1_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block1_2_pad (ZeroPaddin  (None, 6, 6, 512)   0           ['conv5_block1_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv5_blo

 conv5_block3_3_conv (Conv2D)   (None, 4, 4, 2048)   1050624     ['conv5_block3_2_relu[0][0]']    
                                                                                                  
 conv5_block3_out (Add)         (None, 4, 4, 2048)   0           ['conv5_block2_out[0][0]',       
                                                                  'conv5_block3_3_conv[0][0]']    
                                                                                                  
 post_bn (BatchNormalization)   (None, 4, 4, 2048)   8192        ['conv5_block3_out[0][0]']       
                                                                                                  
 post_relu (Activation)         (None, 4, 4, 2048)   0           ['post_bn[0][0]']                
                                                                                                  
 flatten (Flatten)              (None, 32768)        0           ['post_relu[0][0]']              
          

In [56]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 30
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='sound_model_heartresnet.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_valid, y_valid), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/30
59/59 [==============================] - ETA: 0s - loss: 1.4103 - accuracy: 0.5488
Epoch 1: val_loss improved from inf to 0.75752, saving model to sound_model_heartresnet.hdf5
59/59 [==============================] - 130s 2s/step - loss: 1.4103 - accuracy: 0.5488 - val_loss: 0.7575 - val_accuracy: 0.6787
Epoch 2/30
59/59 [==============================] - ETA: 0s - loss: 0.6538 - accuracy: 0.7266
Epoch 2: val_loss improved from 0.75752 to 0.55168, saving model to sound_model_heartresnet.hdf5
59/59 [==============================] - 98s 2s/step - loss: 0.6538 - accuracy: 0.7266 - val_loss: 0.5517 - val_accuracy: 0.7688
Epoch 3/30
59/59 [==============================] - ETA: 0s - loss: 0.5215 - accuracy: 0.7887
Epoch 3: val_loss improved from 0.55168 to 0.52440, saving model to sound_model_heartresnet.hdf5
59/59 [==============================] - 102s 2s/step - loss: 0.5215 - accuracy: 0.7887 - val_loss: 0.5244 - val_accuracy: 0.7958
Epoch 4/30
59/59 [========================

Epoch 29/30
59/59 [==============================] - ETA: 0s - loss: 0.1398 - accuracy: 0.9427  
Epoch 29: val_loss did not improve from 0.20170
59/59 [==============================] - 8087s 139s/step - loss: 0.1398 - accuracy: 0.9427 - val_loss: 0.3994 - val_accuracy: 0.8799
Epoch 30/30
59/59 [==============================] - ETA: 0s - loss: 0.1616 - accuracy: 0.9352
Epoch 30: val_loss did not improve from 0.20170
59/59 [==============================] - 185s 3s/step - loss: 0.1616 - accuracy: 0.9352 - val_loss: 0.2587 - val_accuracy: 0.9189
Training completed in time:  3:43:33.201817
